# Controlable Light Enhencement for Underater Image Enhancement with difusion model
Este notebook foi projetado para testar e avaliar este modelo e as técnicas utilizadas em sua construção. O método utilizado aqui é destinado à pesquisa na área de aprimoramento de imagens. A principal questão é saber se é possível que um modelo de difusão aprenda a melhorar uma imagem subaquática, particularmente suas características de brilho. O objetivo principal é fornecer uma imagem como entrada e um valor de brilho, e o modelo gerará uma nova imagem com iluminação aprimorada. Em outras palavras, o método visa alcançar o aprimoramento controlável da iluminação em toda a imagem.

## Create the necessary envoiriment for the validation




In [ ]:
# make dir data if not exists
# download the dataset using gdown in this link
# descompact 
# move the dataset for the root
# make dir output if not exists
# mke dir ckpt
# make dir UIEB if not exists
# download UIEB ckpt  using gdown in this link
# descompact 
# move for the root
# make dir RUIE if not exists
# download RUIE ckpt using gdown in this link
# descompact
# move for the root
# make dir SUIM if not exists
# download SUIM ckpt using gdown in this link
# descompact
# move for the root

In [ ]:
# Import necessary libraries
import os
import gdown
import zipfile

# Create directories if they don't exist
os.makedirs('data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('ckpt', exist_ok=True)
os.makedirs('UIEB', exist_ok=True)
os.makedirs('RUIE', exist_ok=True)
os.makedirs('SUIM', exist_ok=True)

# Function to download, unzip and move files
def download_and_unzip(url, extract_to, move_to):
    # Download the file
    output_file = os.path.join(extract_to, 'temp.zip')
    gdown.download(url, output_file, quiet=False)
    
    # Unzip the file
    with zipfile.ZipFile(output_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Move the contents to the specified directory
    for item in os.listdir(extract_to):
        item_path = os.path.join(extract_to, item)
        os.rename(item_path, os.path.join(move_to, item))

    # Remove the temporary zip file
    os.remove(output_file)

# Fill in the links for download
dataset_link = 'INSERT_DATASET_LINK_HERE'
uie_b_link = 'INSERT_UIEB_CKPT_LINK_HERE'
ruie_link = 'INSERT_RUIE_CKPT_LINK_HERE'
suim_link = 'INSERT_SUIM_CKPT_LINK_HERE'

# Download, unzip and move the dataset
download_and_unzip(dataset_link, 'data', '/content')

# Download, unzip and move UIEB checkpoint
download_and_unzip(uie_b_link, 'UIEB', '/content')

# Download, unzip and move RUIE checkpoint
download_and_unzip(ruie_link, 'RUIE', '/content')

# Download, unzip and move SUIM checkpoint
download_and_unzip(suim_link, 'SUIM', '/content')

print("All files downloaded and organized successfully.")


## Load The datset data
Give a look

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from src.train import load_data, load_data_test
from src.split_data import load_image_paths
import matplotlib.pyplot as plt
import numpy as np
import torch

def train_dataloader(dataloader, num_images=5):
    # Itera sobre o DataLoader 
    for batch_idx, (data_low, data_high, data_color, data_blur) in enumerate(dataloader):
        if batch_idx >= num_images:
            break
        # Plotagem das imagens
        #Convertendo tensores para NumPy arrays para plotagem
        image_low = np.transpose(data_low.detach().cpu().numpy(), (1, 2, 0))
        image_high = np.transpose(data_high.detach().cpu().numpy(), (1, 2, 0))
        image_color = np.transpose(data_color.detach().cpu().numpy(), (1, 2, 0))
        image_blur = np.transpose(data_blur.detach().cpu().numpy(), (1, 2, 0))
                # Transpose and print shapes
        # print(f"""  data_low: {data_low.shape}
        #                 data_high: {data_high.shape}
        #                 data_color: {data_color.shape}
        #                 data_blur: {data_blur.shape}
        #                 image_low: {image_low.shape}
        #                 image_high: {image_high.shape}
        #                 image_color: {image_color.shape}
        #                 image_blur: {image_blur.shape}
        # """)
        # Mostrando as imagens
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 4, 1)
        plt.imshow(image_low)
        plt.title("Data Low")
        plt.axis('off')

        plt.subplot(1, 4, 2)
        plt.imshow(image_high)
        plt.title("Data High")
        plt.axis('off')

        plt.subplot(1, 4, 3)
        plt.imshow(image_color)
        plt.title("Data Color")
        plt.axis('off')

        plt.subplot(1, 4, 4)
        plt.imshow(image_blur)
        plt.title("Data Blur")
        plt.axis('off')

        plt.show()
def test_dataloader(dataloader, num_images=5):
    # Itera sobre o DataLoader 
    for batch_idx, (data_low, data_high, data_color, data_blur, img) in enumerate(dataloader):
        if batch_idx >= num_images:
            break
        # Plotagem das imagens
        #Convertendo tensores para NumPy arrays para plotagem
        image_low = np.transpose(data_low.detach().cpu().numpy(), (1, 2, 0))
        image_high = np.transpose(data_high.detach().cpu().numpy(), (1, 2, 0))
        image_color = np.transpose(data_color.detach().cpu().numpy(), (1, 2, 0))
        image_blur = np.transpose(data_blur.detach().cpu().numpy(), (1, 2, 0))

                # Transpose and print shapes
        # print(f"""  data_low: {data_low.shape}
        #                 data_high: {data_high.shape}
        #                 data_color: {data_color.shape}
        #                 data_blur: {data_blur.shape}
        #                 image_low: {image_low.shape}
        #                 image_high: {image_high.shape}
        #                 image_color: {image_color.shape}
        #                 image_blur: {image_blur.shape}
        # """)
        # Mostrando as imagens
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 4, 1)
        plt.imshow(image_low)
        plt.title("Data test Low")
        plt.axis('off')

        plt.subplot(1, 4, 2)
        plt.imshow(image_high)
        plt.title("Data test High")
        plt.axis('off')

        plt.subplot(1, 4, 3)
        plt.imshow(image_color)
        plt.title("Data test Color")
        plt.axis('off')

        plt.subplot(1, 4, 4)
        plt.imshow(image_blur)
        plt.title("Data test Blur")
        plt.axis('off')

        plt.show()


# Exemplo de uso:
datapath_train,datapath_test = load_image_paths(dataset_path="data/UDWdata/",dataset="UIEB")
train_loader=load_data(datapath_train, datapath_train)
test_loader = load_data_test(datapath_test,datapath_test)
# Suponha que você tenha um DataLoader chamado 'train_loader' com batch_size=4
train_dataloader(train_loader, num_images=5)
test_dataloader(test_loader, num_images=5)


Acrescentar blur 
funcoes de perda
